In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
# Load Data
cf = pd.read_csv('./data/county_facts.csv')
cfd = pd.read_csv('./data/county_facts_dictionary.csv')
#pr = pd.read_csv('./data/primary_results.csv')
pr = pd.read_csv('./data/scraped_primary_results4.csv')

In [6]:
lname2full = {}
for name in pr['candidate'].unique():
    spl = name.split(' ')
    lname2full[spl[1]] = name

In [4]:
#import traceback
def scrape_primary_results((state, state_abbreviation, party, html_string)):
    soup = BeautifulSoup(html_string)
    rows = []
    counties = soup.find_all("div", { "class" : "race-results--county" })
    for county in counties:
        try:
            county_name = county.find("h2", { "class" : "section-heading" }).text + " County"
            fips = cf[(cf['area_name'] == county_name)]['fips'].iloc[0]
            candidates = county.find_all("div", { "class" : "table__row" })[1:]
            for candidate in candidates:
                candidate_lname = candidate.find("span", { "class" : "candidate-name__last" }).text
                candidate_name = lname2full[candidate_lname]
                candidate_percent = float(candidate.find("div", { "class" : "percent-bar__percent" }).text[:-1])/100
                candidate_votes = int(candidate.find("span", { "class" : "candidate-votes" }).text)
                rows.append([state, state_abbreviation, county_name, fips, party, candidate_name, candidate_votes, candidate_percent])
        except Exception as e:
            if type(e) != AttributeError:
                pass
                #print e
                #traceback.print_exc()
                
    columns = ["state", "state_abbreviation", "county", "fips", "party", "candidate", "votes", "fraction_votes"]
    if len(rows) == 0:
        print "Bad Data for: " + state + " " + party
        return None
    else:
        return pd.DataFrame(rows, columns=columns)

In [6]:
with open('./automation/scraped_data.txt', 'r') as f:
    text = f.readline()
    my_data = None
    while text != '':
        scrape = eval(text)
        print scrape[0]
        
        results = scrape_primary_results(scrape)
        if results is not None:
            if my_data is None:
                my_data = results
            else:
                my_data = my_data.append(results)
    
        text = f.readline()

Hawaii
Idaho
Michigan


/usr/local/lib/python2.7/dist-packages/pandas/core/ops.py:562: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  result = lib.scalar_compare(x, y, op)


In [7]:
my_data.to_csv('./data/scraped_primary_results3.csv')